In [1]:
import numpy as np
import os
import librosa
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.layers import PReLU
from scipy.signal import butter, lfilter, freqz
from sigma import get_glottal
from time import time
from numpy.lib.stride_tricks import sliding_window_view
import warnings
warnings.filterwarnings('ignore')

In [10]:
model = keras.models.load_model(r'C:\Users\user\Desktop\segan_model_7')

In [3]:
def reconstruct_egg(speech):
    
    dataset = sliding_window_view(speech, 2048)[::1024, :]
    
    n = int((len(speech) - 2048)/1024) + 1
    
    egg_windows = model.predict(dataset)[:,:,0]
    
    hann = 0.5*(1 - np.cos(np.pi*np.arange(2048)/1024))
    
    egg_windows[1:-1] = egg_windows[1:-1]*hann
    
    egg_windows[0,1024:] = egg_windows[0,1024:]*hann[1024:]
    
    egg_windows[-1,:1024] = egg_windows[-1,:1024]*hann[:1024]
    
    egg_reconstructed = np.zeros(2048 + (n-1)*1024)
    
    for i in range(n):
        egg_reconstructed[1024*i:1024*i+2048] = egg_reconstructed[1024*i:1024*i+2048] + egg_windows[i]
        
    return egg_reconstructed

In [4]:
def naylor_metrics(ref_signal, est_signal):

    assert (np.squeeze(ref_signal).ndim == 1)
    assert (np.squeeze(est_signal).ndim == 1)

    ref_signal = np.squeeze(ref_signal)
    est_signal = np.squeeze(est_signal)

    min_f0 = 50
    max_f0 = 500
    min_glottal_cycle = 1 / max_f0
    max_glottal_cycle = 1 / min_f0

    nHit = 0
    nMiss = 0
    nFalse = 0
    nCycles = 0
    highNumCycles = 100000
    estimation_distance = np.full(highNumCycles, np.nan)

    ref_fwdiffs = np.diff(ref_signal)[1:]
    ref_bwdiffs = np.diff(ref_signal)[:-1]

    for i in range(len(ref_fwdiffs)):
        ref_cur_sample = ref_signal[i + 1]
        ref_dist_fw = ref_fwdiffs[i]
        ref_dist_bw = ref_bwdiffs[i]

        dist_in_allowed_range = min_glottal_cycle <= ref_dist_fw <= max_glottal_cycle and min_glottal_cycle <= ref_dist_bw <= max_glottal_cycle
        if dist_in_allowed_range:

            cycle_start = ref_cur_sample - ref_dist_bw / 2
            cycle_stop = ref_cur_sample + ref_dist_fw / 2

            est_GCIs_in_cycle = est_signal[np.logical_and(est_signal > cycle_start, est_signal < cycle_stop)]
            n_est_in_cycle = np.count_nonzero(est_GCIs_in_cycle)

            nCycles += 1

            if n_est_in_cycle == 1:
                nHit += 1
                estimation_distance[nHit] = est_GCIs_in_cycle[0] - ref_cur_sample
            elif n_est_in_cycle < 1:
                nMiss += 1
            else:
                nFalse += 1

    estimation_distance = estimation_distance[np.invert(np.isnan(estimation_distance))]
    
    try:
        identification_rate = nHit / nCycles
    except:
        return {'identification_rate' : -1}
    else:
        identification_rate = nHit / nCycles
        miss_rate = nMiss / nCycles
        false_alarm_rate = nFalse / nCycles
        identification_accuracy = 0 if np.size(estimation_distance) == 0 else np.std(estimation_distance)
    

        return {
            'identification_rate': identification_rate,
            'miss_rate': miss_rate,
            'false_alarm_rate': false_alarm_rate,
            'identification_accuracy': identification_accuracy
        }


In [5]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [22]:
dataset = r'C:\Users\user\Desktop\dataset'

identification_rate_gci = 0
miss_rate_gci = 0
false_alarm_rate_gci = 0
identification_accuracy_gci = 0

identification_rate_goi = 0
miss_rate_goi = 0
false_alarm_rate_goi = 0
identification_accuracy_goi = 0

n = 0

t1 = time()
k = os.listdir(dataset + r"\speech")
np.random.shuffle(k)
for i in k:
    if i[:3] == "bdl":

        n += 1

        egg,sr_egg = librosa.load(dataset + r"\egg" + "\\" + i)
        speech,_ = librosa.load(dataset + r"\speech" + "\\" + i)

        egg_recreated = reconstruct_egg(speech)
        egg = egg[:len(egg_recreated)]

        egg_low_pass = butter_lowpass_filter(egg_recreated, cutoff = 1000, fs = sr_egg, order = 6)

        gci,goi = get_glottal(egg,sr_egg)
        gci_rec,goi_rec = get_glottal(egg_low_pass,sr_egg)

        maximum = np.amax((np.amax(gci),np.amax(gci_rec)))
        val = naylor_metrics(gci/maximum, gci_rec/maximum)
        
        if val["identification_rate"] == -1:
            n -= 1
            continue

        identification_rate_gci += val["identification_rate"]
        miss_rate_gci += val["miss_rate"]
        false_alarm_rate_gci += val["false_alarm_rate"]
        identification_accuracy_gci += val["identification_accuracy"]

        maximum = np.amax((np.amax(goi),np.amax(goi_rec)))
        val = naylor_metrics(goi/maximum, goi_rec/maximum)
        
        if val["identification_rate"] == -1:
            n -= 1
            continue

        identification_rate_goi += val["identification_rate"]
        miss_rate_goi += val["miss_rate"]
        false_alarm_rate_goi += val["false_alarm_rate"]
        identification_accuracy_goi += val["identification_accuracy"]

        if n==50:
            break
        
for i in k:
    if i[:3] == "jmk":

        n += 1

        egg,sr_egg = librosa.load(dataset + r"\egg" + "\\" + i)
        speech,_ = librosa.load(dataset + r"\speech" + "\\" + i)

        egg_recreated = reconstruct_egg(speech)
        egg = egg[:len(egg_recreated)]

        egg_low_pass = butter_lowpass_filter(egg_recreated, cutoff = 1000, fs = sr_egg, order = 6)

        gci,goi = get_glottal(egg,sr_egg)
        gci_rec,goi_rec = get_glottal(egg_low_pass,sr_egg)

        maximum = np.amax((np.amax(gci),np.amax(gci_rec)))
        val = naylor_metrics(gci/maximum, gci_rec/maximum)
        
        if val["identification_rate"] == -1:
            n -= 1
            continue

        identification_rate_gci += val["identification_rate"]
        miss_rate_gci += val["miss_rate"]
        false_alarm_rate_gci += val["false_alarm_rate"]
        identification_accuracy_gci += val["identification_accuracy"]

        maximum = np.amax((np.amax(goi),np.amax(goi_rec)))
        val = naylor_metrics(goi/maximum, goi_rec/maximum)
        
        if val["identification_rate"] == -1:
            n -= 1
            continue

        identification_rate_goi += val["identification_rate"]
        miss_rate_goi += val["miss_rate"]
        false_alarm_rate_goi += val["false_alarm_rate"]
        identification_accuracy_goi += val["identification_accuracy"]

        if n==100:
            break
            
for i in k:
    if i[:3] == "slt":

        n += 1

        egg,sr_egg = librosa.load(dataset + r"\egg" + "\\" + i)
        speech,_ = librosa.load(dataset + r"\speech" + "\\" + i)

        egg_recreated = reconstruct_egg(speech)
        egg = egg[:len(egg_recreated)]

        egg_low_pass = butter_lowpass_filter(egg_recreated, cutoff = 1000, fs = sr_egg, order = 6)

        gci,goi = get_glottal(egg,sr_egg)
        gci_rec,goi_rec = get_glottal(egg_low_pass,sr_egg)

        maximum = np.amax((np.amax(gci),np.amax(gci_rec)))
        val = naylor_metrics(gci/maximum, gci_rec/maximum)
        
        if val["identification_rate"] == -1:
            n -= 1
            continue

        identification_rate_gci += val["identification_rate"]
        miss_rate_gci += val["miss_rate"]
        false_alarm_rate_gci += val["false_alarm_rate"]
        identification_accuracy_gci += val["identification_accuracy"]

        maximum = np.amax((np.amax(goi),np.amax(goi_rec)))
        val = naylor_metrics(goi/maximum, goi_rec/maximum)
        
        if val["identification_rate"] == -1:
            n -= 1
            continue

        identification_rate_goi += val["identification_rate"]
        miss_rate_goi += val["miss_rate"]
        false_alarm_rate_goi += val["false_alarm_rate"]
        identification_accuracy_goi += val["identification_accuracy"]

        if n==150:
            break
    
identification_rate_gci /= n 
miss_rate_gci /= n 
false_alarm_rate_gci /= n 
identification_accuracy_gci /= n 

identification_rate_goi /= n 
miss_rate_goi /= n 
false_alarm_rate_goi /= n 
identification_accuracy_goi /= n

print("total time",time() - t1)

total time 235.17610239982605


In [23]:
print("GCI")
print("Identification Rate: ",identification_rate_gci)
print("Miss Rate: ",miss_rate_gci)
print("False Alarm Rate: ",false_alarm_rate_gci)
print("Identification Accuracy: ",identification_accuracy_goi)
print("GOI")
print("Identification Rate: ",identification_rate_goi)
print("Miss Rate: ",miss_rate_goi)
print("False Alarm Rate: ",false_alarm_rate_goi)
print("Identification Accuracy: ",identification_accuracy_goi)

GCI
Identification Rate:  0.93781897557351
Miss Rate:  0.03503558976870273
False Alarm Rate:  0.040478767991120254
Identification Accuracy:  0.0003761602268740387
GOI
Identification Rate:  0.9221733773826671
Miss Rate:  0.05425677131613817
False Alarm Rate:  0.023569851301194114
Identification Accuracy:  0.0003761602268740387
